# Business Cycle - Equities

## Imports

In [273]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import pmp_functions_v4 as pmp

import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

path = "../../Data_Ryan"

## Data Cleaning

In [274]:
# --- Load Riskfree Rate ---
factors_data = pd.read_excel(
    f"{path}/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100

riskfree = factors_data["RF"].resample('ME').last()
display(riskfree)

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_51076/711168680.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


1926-07-31    0.0022
1926-08-31    0.0025
1926-09-30    0.0023
1926-10-31    0.0032
1926-11-30    0.0031
               ...  
2025-06-30    0.0034
2025-07-31    0.0034
2025-08-31    0.0038
2025-09-30    0.0033
2025-10-31    0.0037
Freq: ME, Name: RF, Length: 1192, dtype: float64

In [275]:
# --- Load Factors Data ---
famafrench_data = pd.read_csv(
    f"{path}/famafrench_factors.csv",
    index_col = 0,
    parse_dates = True
)

famafrench_data.index = pd.to_datetime(famafrench_data.index, format='%Y%m')
famafrench_data.index = famafrench_data.index + pd.offsets.MonthEnd(0)
famafrench_data /= 100
display(famafrench_data)

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_51076/2041547973.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  famafrench_data = pd.read_csv(


,MKT-RF,SMB,HML,RMW,CMA,UMD,BAB
DATE,,,,,,,
1980-01-31,0.000550,0.000188,0.000185,-0.000184,0.000189,0.000745,0.000695
1980-02-29,-0.000123,-0.000162,0.000059,-0.000095,0.000292,0.000789,-0.000132
1980-03-31,-0.001289,-0.000697,-0.000096,0.000182,-0.000105,-0.000958,-0.001181
1980-04-30,0.000396,0.000105,0.000103,-0.000218,0.000034,-0.000048,0.000574
1980-05-31,0.000526,0.000200,0.000038,0.000043,-0.000063,-0.000118,0.000618
...,...,...,...,...,...,...,...
2025-06-30,0.000486,-0.000002,-0.000160,-0.000320,0.000145,-0.000264,0.000527
2025-07-31,0.000198,-0.000015,-0.000127,-0.000029,-0.000208,-0.000096,0.000184
2025-08-31,0.000185,0.000488,0.000442,-0.000068,0.000207,-0.000354,0.000646


In [276]:
# --- Benchmark Data ---
benchmark_data = pd.read_excel(
    f"{path}/Benchmarks.xlsx",
    index_col = 0,
    parse_dates = True
)
display(benchmark_data)
benchmark_data.index = pd.to_datetime(benchmark_data.index)
benchmark_data = benchmark_data.resample('ME').last()

benchmark_TR = benchmark_data[['MSCI World']].pct_change()
benchmark_XR = benchmark_TR.sub(riskfree, axis = 0)

benchmark_TR.columns = ['Benchmark Total Return']
benchmark_XR.columns = ['Benchmark Excess Return']
benchmark_returns = pd.concat([benchmark_TR, benchmark_XR], axis = 1).dropna()

display(benchmark_returns)

,S&P 500,MSCI World
Date,,
1986-12-31,242.1700,NaN
1987-01-30,274.7800,NaN
1987-02-27,285.6377,NaN
1987-03-31,293.8792,NaN
1987-04-30,291.2698,NaN
...,...,...
2025-07-31,14412.5527,8214.1572
2025-08-29,14704.7217,8431.0801
2025-09-30,15240.0381,8705.7139


,Benchmark Total Return,Benchmark Excess Return
1991-01-31,0.034364,0.029164
1991-02-28,0.090450,0.085650
1991-03-31,-0.031331,-0.035731
1991-04-30,0.005910,0.000610
1991-05-31,0.020740,0.016040
...,...,...
2025-06-30,0.043488,0.040088
2025-07-31,0.013121,0.009721
2025-08-31,0.026408,0.022608
2025-09-30,0.032574,0.029274


In [277]:
# --- Load Macro Data ---
CPI_forecasts = pd.read_excel(
    f"{path}/Inflation_forecasts.xlsx",
    index_col = 0,
    parse_dates = True
)
CPI_forecasts.index = pd.to_datetime(CPI_forecasts.index)
CPI_forecasts.index = CPI_forecasts.index + pd.offsets.MonthEnd(0)
CPI_forecasts *= 100

display(CPI_forecasts)

RGDP_forecasts = pd.read_excel(
    f"{path}/RGDP_forecasts.xlsx",
    index_col = 0,
    parse_dates = True
)
RGDP_forecasts.index = pd.to_datetime(RGDP_forecasts.index)
RGDP_forecasts.index = RGDP_forecasts.index + pd.offsets.MonthEnd(0)
RGDP_forecasts *= 100

display(RGDP_forecasts)

,UK,CH,JP,AU,EU,US
Date,,,,,,
1970-01-31,4.95739,2.31338,7.75127,NaN,NaN,NaN
1970-02-28,4.93065,2.12360,7.75127,NaN,NaN,NaN
1970-03-31,5.14198,2.49918,7.71710,NaN,NaN,NaN
1970-04-30,5.61884,2.59281,7.68362,NaN,NaN,NaN
1970-05-31,6.08365,3.13683,7.24558,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,4.10000,0.10000,3.30000,2.1,1.96760,3.17
2025-07-31,4.20000,0.20000,3.10000,3.2,2.00791,2.79
2025-08-31,4.10000,0.20000,2.70000,3.2,2.02889,2.79


,AU,UK,CH,JP,EU,US
Date,,,,,,
1970-07-31,3.15789,NaN,NaN,NaN,NaN,2.936175
1970-08-31,3.15789,NaN,NaN,NaN,NaN,2.936175
1970-09-30,3.15789,NaN,NaN,NaN,NaN,2.936175
1970-10-31,5.26317,NaN,NaN,NaN,NaN,2.994250
1970-11-30,5.26317,NaN,NaN,NaN,NaN,2.994250
...,...,...,...,...,...,...
2025-06-30,2.10000,2.5,2.439516,1.976375,1.522007,1.345350
2025-07-31,3.20000,2.5,1.284580,1.074700,1.350512,1.450175
2025-08-31,3.20000,2.5,1.284580,1.074700,1.350512,1.450175


In [278]:
# --- Load Equity Price Data ---
equity_prices = pd.read_excel(
    f"{path}/Equity Data.xlsx",
    index_col = 0,
    parse_dates = True
)
equity_prices.index = pd.to_datetime(equity_prices.index)
equity_prices.index = equity_prices.index + pd.offsets.MonthEnd(0)

display(equity_prices)

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1997-09-30,1206.821289,NaN,NaN,7.33429,7487.55371,NaN,NaN
1997-10-31,1168.258667,NaN,NaN,6.82642,7084.46436,NaN,NaN
1997-11-30,1207.453491,NaN,NaN,6.34016,7152.55615,NaN,NaN
1997-12-31,1223.840210,NaN,NaN,5.86087,7365.37451,NaN,NaN
1998-01-31,1234.778442,NaN,NaN,6.36481,7690.75049,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,6308.888672,5641.31592,14994.83984,19.64406,12121.12598,14955.800000,6276.48438
2025-07-31,6430.451172,5627.64307,14552.98438,19.41757,12131.93066,15094.600000,6109.94238
2025-08-31,6529.819336,5881.38037,15274.74121,20.75129,12521.77637,15223.700000,6284.96094


## Global Variables

In [279]:
frequency = 1
t_cost = 0
# window = 12*20
short = True
beta_neutral = False
target_vol = 0.10
rf = riskfree
benchmark = benchmark_data

## Signal Generation

In [280]:
# --- Compute Business Cycle Signal ---
CPI_component = CPI_forecasts.diff(12)
RGDP_component = RGDP_forecasts.diff(12)

display(CPI_component)
display(RGDP_component)

# # Reindex RGDP trend to match CPI trend index exactly
# RGDP_component = RGDP_trend.reindex(CPI_trend.index).ffill()
# CPI_component = CPI_trend

,UK,CH,JP,AU,EU,US
Date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,1.31793,-1.22807,0.5,-1.71445,-0.54354,0.39
2025-07-31,1.17674,-1.09059,0.3,0.39146,-0.57075,0.39
2025-08-31,1.00881,-0.86037,-0.3,0.39146,-0.12434,0.39


,AU,UK,CH,JP,EU,US
Date,,,,,,
1970-07-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-08-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-09-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-10-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-11-30,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,-1.71445,0.7,2.356965,2.957250,0.993241,-0.529850
2025-07-31,0.39146,1.7,-0.644766,0.676316,0.386473,-0.337325
2025-08-31,0.39146,1.7,-0.644766,0.676316,0.386473,-0.337325


In [281]:
# --- Composite Signal Construction ---
# Logic: For Equities, we want High Growth (+) and Low Inflation (-)
# We combine them: 50% Growth, 50% Inverse Inflation
business_cyle_signal = (0.5 * RGDP_component) - (0.5 * CPI_component)
business_cyle_signal = business_cyle_signal.resample('ME').last()

display("Business Cycle Signal:")
display(business_cyle_signal)

'Business Cycle Signal:'

,AU,CH,EU,JP,UK,US
Date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,0.0,1.792518,0.768391,1.228625,-0.308965,-0.459925
2025-07-31,0.0,0.222912,0.478611,0.188158,0.261630,-0.363662
2025-08-31,0.0,0.107802,0.255406,0.488158,0.345595,-0.363662


## Asset Returns

In [282]:
# --- Equity Returns ---
equity_XR = equity_prices.pct_change()
equity_XR = equity_XR.resample('ME').last()

display(equity_XR)

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1997-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-31,-0.031954,NaN,NaN,-0.069246,-0.053835,NaN,NaN
1997-11-30,0.033550,NaN,NaN,-0.071232,0.009611,NaN,NaN
1997-12-31,0.013571,NaN,NaN,-0.075596,0.029754,NaN,NaN
1998-01-31,0.008938,NaN,NaN,0.085984,0.044176,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,0.047850,0.034649,0.008742,0.018397,0.017408,0.087457,0.024923
2025-07-31,0.019268,-0.002424,-0.029467,-0.011530,0.000891,0.009281,-0.026534
2025-08-31,0.015453,0.045088,0.049595,0.068686,0.032134,0.008553,0.028645


## Portfolio Construction

In [283]:
# --- Ranking & Weighting ---
# Rank countries 1 to N for each month based on the raw signal.
# axis = 1 means we rank across columns (countries).

rank = business_cyle_signal.rank(axis = 1, method = 'average', ascending = False)
display(rank)

,AU,CH,EU,JP,UK,US
Date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,4.0,1.0,3.0,2.0,5.0,6.0
2025-07-31,5.0,3.0,1.0,4.0,2.0,6.0
2025-08-31,5.0,4.0,3.0,1.0,2.0,6.0


In [284]:
# --- Standardize Ranks ---
# Convert ranks into Z-scores (Weights) that sum to zero.
# Weight = (Rank - Mean_Rank) / Std_Dev_Rank

rank_mean = rank.mean(axis = 1)
rank_stds = rank.std(axis = 1)
standardized_weights = rank.sub(rank_mean, axis = 0).div(rank_stds, axis = 0)

display(standardized_weights)

,AU,CH,EU,JP,UK,US
Date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,0.267261,-1.336306,-0.267261,-0.801784,0.801784,1.336306
2025-07-31,0.801784,-0.267261,-1.336306,0.267261,-0.801784,1.336306
2025-08-31,0.801784,0.267261,-0.267261,-1.336306,-0.801784,1.336306


In [285]:
# --- Volatility Scaling (Risk Management) ---
# Step A: Calculate 'Raw' Strategy Returns (Before Vol Scaling)
# IMPORTANT: Shift weights by 1 to trade next month's return.
strategy_raw_ret = (standardized_weights.shift(1) * equity_XR).sum(axis=1)

# Step B: Forecast Volatility
# Calculate realized volatility over a 36-month rolling window (annualized)
# We use the raw strategy's realized vol to estimate future volatility.
expected_vol = strategy_raw_ret.rolling(window = 36).std() * np.sqrt(12)

# We use previous rolling volatility (shift 1) to size today's position
lev_factor = target_vol / expected_vol.shift(1)

## **⭐ CRITICAL CHANGE: Scaling the Weights**

# Step C: Estimate Portfolio Weights
# Apply the leverage factor to the standardized weights
# We use .mul(axis=0) to multiply the 2D DataFrame (weights) 
# by the 1D Series (lev_factor_series) along the rows (axis=0).
final_strategy_weights = standardized_weights.mul(lev_factor, axis=0).fillna(0).loc['1998-01-31':'2025-10-31']

display(final_strategy_weights)

,AU,CH,EU,JP,UK,US
Date,,,,,,
1998-01-31,0.751887,-1.503774,-0.751887,1.503774,0.000000,0.000000
1998-02-28,0.612914,-1.225827,-0.612914,1.225827,0.000000,0.000000
1998-03-31,0.606210,-1.212420,-0.606210,1.212420,0.000000,0.000000
1998-04-30,0.545422,-1.090845,0.000000,1.090845,0.000000,-0.545422
1998-05-31,0.543201,-1.086401,0.000000,1.086401,0.000000,-0.543201
...,...,...,...,...,...,...
2025-06-30,0.143949,-0.719746,-0.143949,-0.431847,0.431847,0.719746
2025-07-31,0.428194,-0.142731,-0.713656,0.142731,-0.428194,0.713656
2025-08-31,0.418812,0.139604,-0.139604,-0.698020,-0.418812,0.698020


## Backtest

In [286]:
benchmark_TR = benchmark_TR.squeeze()
benchmark_XR = benchmark_XR.squeeze()

In [287]:
results = pmp.run_cc_strategy(
    weights = final_strategy_weights,
    returns = equity_XR,
    rf = riskfree,
    frequency = frequency,
    t_cost = t_cost, 
    benchmark = benchmark_XR,
    long_short = short,
    beta_neutral = beta_neutral
)

display(results)

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,
1998-02-28,0.026700,0.026700,0.064065,0.000000,0.0,0.0039,0.328430,-0.665686,-0.334314,0.671570,0.000000,0.000000
1998-03-31,-0.088859,-0.088859,0.039375,0.838741,0.0,0.0039,0.353968,-0.670794,-0.329206,0.646032,0.000000,0.000000
1998-04-30,-0.024820,-0.024820,0.005630,0.818630,0.0,0.0043,0.339059,-0.667812,-0.332188,0.660941,0.000000,0.000000
1998-05-31,-0.024982,-0.024982,-0.015149,0.968455,0.0,0.0040,0.342675,-0.673711,0.000000,0.657325,0.000000,-0.326289
1998-06-30,-0.013227,-0.013227,0.020679,0.629602,0.0,0.0041,0.332592,-0.659147,0.000000,0.667408,0.000000,-0.340853
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,0.028375,0.028375,0.040088,0.262341,0.0,0.0034,0.108502,-0.550400,-0.114758,-0.334842,0.327210,0.564287
2025-07-31,0.043912,0.043912,0.009721,0.295542,0.0,0.0034,0.105851,-0.548241,-0.114524,-0.337235,0.330032,0.564117
2025-08-31,-0.001143,-0.001143,0.022608,1.357358,0.0,0.0038,0.337806,-0.112896,-0.553772,0.115239,-0.333333,0.546955


## Performance Statistics

In [288]:
pmp.run_perf_summary_benchmark_vs_strategy(results, alreadyXs = True)

,Benchmark,Strategy
Arithm Avg Total Return,6.9887,1.9753
Arithm Avg Xs Return,4.9498,-0.0636
Std Xs Returns,15.5470,11.2971
Sharpe Arithmetic,0.3184,-0.0056
Geom Avg Total Return,5.9249,1.3536
Geom Avg Xs Return,3.8686,-0.7028
Sharpe Geometric,0.2488,-0.0622
Min Xs Return,-19.0940,-14.2192
Max Xs Return,12.8084,13.2240
Skewness,-0.6049,-0.0678
